In [2]:
!pip install torch

     |████████████████████████████████| 776.3 MB 700 bytes/s  0:00:016


In [27]:
import torch
from torch.autograd import Variable

x = torch.Tensor(2, 3)
x

tensor([[-5.4076e-23,  3.0766e-41, -7.3267e-15],
        [ 4.5633e-41,  4.4842e-44,  0.0000e+00]])

In [7]:
x = torch.rand(2, 3)
x

tensor([[0.6569, 0.5412, 0.2112],
        [0.8432, 0.2255, 0.1807]])

In [25]:
y = torch.ones(2, 3)
y = y*2
y[:,1] += 1
y

tensor([[2., 3., 2.],
        [2., 3., 2.]])

In [34]:
x = Variable(torch.ones(2, 2)*2, requires_grad = True)
z = 2 * (x * x) + 5 * x
z.backward(torch.ones(2, 2))
x.grad

tensor([[13., 13.],
        [13., 13.]])

# Простейшая нейронная сеть на датасете MNIST

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
def simple_gradient():
    # print the gradient of 2x^2 + 5x
    x = Variable(torch.ones(2, 2) * 2, requires_grad=True)
    z = 2 * (x * x) + 5 * x
    # run the backpropagation
    z.backward(torch.ones(2, 2))
    print(x.grad)

In [9]:
def create_nn(batch_size=200, learning_rate=0.01, epochs=3, log_interval=10):

    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=batch_size, shuffle=True)
    
    
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(28 * 28, 200)
            self.fc2 = nn.Linear(200, 200)
            self.fc3 = nn.Linear(200, 10)

        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return F.log_softmax(x)
        
        
    net = Net()
    print(net)

    # create a stochastic gradient descent optimizer
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    # create a loss function
    criterion = nn.NLLLoss()
    
    
     # run the main training loop
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data), Variable(target)
            # resize data from (batch_size, 1, 28, 28) to (batch_size, 28*28)
            data = data.view(-1, 28*28)
            optimizer.zero_grad()
            net_out = net(data)
            loss = criterion(net_out, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.item()))
                
    
    
    # run a test loop
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        data = data.view(-1, 28 * 28)
        net_out = net(data)
        # sum up batch loss
        test_loss += criterion(net_out, target).item()
        pred = net_out.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
if __name__ == "__main__":
    run_opt = 2
    if run_opt == 1:
        simple_gradient()
    elif run_opt == 2:
        create_nn()

Net(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.302495


<ipython-input-9-60c11e94d64a>:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 0 [2000/60000 (3%)]	Loss: 2.211121
Train Epoch: 0 [4000/60000 (7%)]	Loss: 1.942721
Train Epoch: 0 [6000/60000 (10%)]	Loss: 1.514886
Train Epoch: 0 [8000/60000 (13%)]	Loss: 0.830566
Train Epoch: 0 [10000/60000 (17%)]	Loss: 0.585620
Train Epoch: 0 [12000/60000 (20%)]	Loss: 0.429515
Train Epoch: 0 [14000/60000 (23%)]	Loss: 0.542018
Train Epoch: 0 [16000/60000 (27%)]	Loss: 0.347749
Train Epoch: 0 [18000/60000 (30%)]	Loss: 0.442372
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.287191
Train Epoch: 0 [22000/60000 (37%)]	Loss: 0.310567
Train Epoch: 0 [24000/60000 (40%)]	Loss: 0.353946
Train Epoch: 0 [26000/60000 (43%)]	Loss: 0.290606
Train Epoch: 0 [28000/60000 (47%)]	Loss: 0.318203
Train Epoch: 0 [30000/60000 (50%)]	Loss: 0.193455
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.348322
Train Epoch: 0 [34000/60000 (57%)]	Loss: 0.373362
Train Epoch: 0 [36000/60000 (60%)]	Loss: 0.332179
Train Epoch: 0 [38000/60000 (63%)]	Loss: 0.250362
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.327004
Train 

<ipython-input-9-60c11e94d64a>:64: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test set: Average loss: 0.0006, Accuracy: 9634/10000 (96%)

